In [ ]:
"""
When a graph contains negative edges (moving from one node to another is a profit), Dijkstra's algorithm cannot ensure to return the optimal paths. Dijkstra is greedy, it assumes that a visited node has a correctly calculated path already. but with negative edges this assumption is false.

We need a brute force approach here - comparing every possible connection for as many nodes we have. That's the Bellman-Ford algorithm.

If we can still make optimizations on the final pass, we know to be in a downward spiral.
"""

In [ ]:
def bellman_ford(graph, start_label):
    """
    Compute shortest paths from 'start_label' in a directed graph
    that may have negative edges.

    Returns:
    {
      'paths': {
         label: { 'cost': float or None, 'path': [list_of_vertex_labels] }
      },
      'negative_cycle': bool
    }
    If 'negative_cycle' is True, the path costs may not be correct
    for vertices involved in that cycle.
    """

    # Extract metadata from the graph object
    labels = graph.labels
    n = graph.num_vertices

    # dist[i] stores the minimum cost found so far from start to node i
    dist = [float("inf")] * n
    # prev[i] stores the index of the node immediately preceding node i
    prev = [None] * n

    # Find the integer index of our starting string label
    start_index = labels.index(start_label)
    # Distance from start to itself is always 0
    dist[start_index] = 0

    # Outer loop: The maximum length of a simple path is n-1 edges.
    # Relaxing all edges n-1 times guarantees finding the shortest path.
    for _ in range(n - 1):
        # Iterate through every possible 'source' vertex u
        for u in range(n):
            # If u is unreachable, we can't use it to find a path to its neighbors
            if dist[u] == float("inf"):
                continue

            # Iterate through every possible 'destination' vertex v
            for v in range(n):
                weight = graph.adj_matrix[u][v]

                # Check if an edge exists (using 'is not None' to allow 0-weight edges)
                if weight is not None:
                    new_dist = dist[u] + weight

                    # If the path through u is cheaper than the best known path to v
                    if new_dist < dist[v]:
                        dist[v] = new_dist  # Update the minimum distance
                        prev[v] = u  # Record u as the parent of v

    # Final Pass: Check for negative cycles.
    # If we can still relax an edge on the n-th pass, a negative cycle exists.
    negative_cycle = False
    for u in range(n):
        if dist[u] == float("inf"):
            continue
        for v in range(n):
            weight = graph.adj_matrix[u][v]
            # If a shorter path is found now, the cost is actually -infinity
            if weight is not None and dist[u] + weight < dist[v]:
                negative_cycle = True
                break
        if negative_cycle:
            break

    # Build result
    result = {"paths": {}, "negative_cycle": negative_cycle}

    # Reconstruct each vertex's path from start, if reachable
    for i, label in enumerate(labels):
        if dist[i] == float("inf"):
            # Unreachable
            cost = None
            path = []
        elif negative_cycle:
            # If there's a cycle, we can't guarantee a simple path exists via 'prev'
            cost, path = dist[i], ["Error: Negative Cycle Detected"]
        else:
            cost = dist[i]
            # Reconstruct path by walking backwards via 'prev'
            path_indices = []
            current = i
            while current is not None:
                path_indices.append(current)
                current = prev[current]
            path_indices.reverse()  # now it's from start_index -> i

            path = [labels[idx] for idx in path_indices]

        result["paths"][label] = {"cost": cost, "path": path}

    return result